# Student name: Veselova Arina

## Predicting

In [305]:
import pandas as pd
# download data
data_long = pd.read_csv('long.csv')
data_wide = pd.read_csv('wide.csv')

In [306]:
pd.set_option('display.max_columns', None)

In [307]:
data_wide

,session,lot_order,lot_order_norm,lot_num,lot_id,estim,res_i,result,win_bid,win_firm,neg_bid,neg_firm,neg_delay,stand,cut,surf,land_area,forest,clauses,quality,conditions,grape,ref,owner,n_trees,n_poles,v_total,v_crown,v_stem,v_stump,v_stem_other_foliar,v_stem_other_conifer,v_stem_oak,v_stem_spruce,v_stem_beech,v_stem_pine,v_stem_fir,v_crown_foliar,v_crown_conifer,res_im1,res_im2,res_im3,date_im1,date_im2,date_im3,n_bidders,n_lots,all_bids,all_firms,random_bid_1,random_bid_2,random_bid_3,random_bid_4,random_bid_5,random_bid_6,herf1,herf100,ln_herf100,min_bid,max_bid,mean_bid,sd_bid,sec_bid,thi_bid,dres_min_bid,dres_mean_bid,dres_max_bid,pres_min_bid,pres_mean_bid,pres_max_bid,min_resest,mgmin,max_resest,mgmax,mgr,mge,rge,revise_up,revise_down,ln_session,ln_lot_order,ln_lot_order_norm,ln_surf,ln_n_bidders,ln_n_trees,ln_n_poles,ln_v_total,ln_v_crown,ln_v_stem,ln_v_stump,ln_v_stem_other_foliar,ln_v_stem_other_conifer,ln_v_stem_oak,ln_v_stem_spruce,ln_v_stem_beech,ln_v_stem_pine,ln_v_stem_fir,ln_v_crown_foliar,ln_v_crown_conifer,ln_res_i,ln_res_im1,ln_res_im2,ln_res_im3,ln_win_bid,ln_estim,ln_max_bid,ln_min_bid,ln_mean_bid,ln_sec_bid,ln_thi_bid,ln_sd_bid,ln_dres_min_bid,ln_dres_max_bid,ln_dres_mean_bid,ln_pres_min_bid,ln_pres_max_bid,ln_pres_mean_bid,ln_random_bid_1,ln_random_bid_2,ln_random_bid_3,ln_random_bid_4,ln_random_bid_5,ln_random_bid_6
0,1,1,0.008197,35069,03200400A7,25323,24000,O,23069.0,11.0,NaN,NaN,NaN,C,R,12.01,A,AUDUND,C,B,DD,M0,NaN,DO,346,255,933.14,374.54,558.60,0.0,186.22,0.0,144.96,0.00,227.42,0.00000,0.00,374.54,0.00,NaN,NaN,NaN,NaN,NaN,NaN,6,122,23069 & 22200 & 20807 & 20512 & 17666 & 15304,11 & 45 & 5 & 48 & 31 & 3,22200.0,23069.0,20807.0,17666.0,15304.0,20512.0,0.344229,34.422903,3.567359,15304.0,23069.0,19926.333333,2920.259075,22200.0,20807.0,8696.0,4073.666667,931.0,0.362333,0.169736,0.038792,24000,0.0,25323,0.0,0.0,0.0,0,NaN,1.0,0.693147,0.693147,0.008163,2.565718,1.945910,5.849325,5.545177,6.839626,5.928365,6.327222,0.0,5.232284,0.000000,4.983333,0.000000,5.431186,0.00000,0.000000,5.928365,0.000000,10.085851,NaN,NaN,NaN,10.046288,10.139508,10.046288,9.635935,9.899848,10.007893,9.943093,7.979770,9.070733,6.837333,8.312544,0.309199,0.038058,0.156778,10.007893,10.046288,9.943093,9.779454,9.635935,9.928814
1,1,2,0.016393,35070,03200200A7,9203,9000,O,8400.0,131.0,NaN,NaN,NaN,C,R,7.38,NArr,AUDUND,C,B,ADF,M0,NaN,DO,124,80,388.30,166.34,221.96,0.0,27.35,4.0,38.82,9.87,141.92,0.00000,0.00,166.34,0.00,NaN,NaN,NaN,NaN,NaN,NaN,5,122,8400 & 7920 & 7912 & 6346 & 6312,131 & 3 & 11 & 106 & 48,8400.0,7920.0,6346.0,6312.0,NaN,NaN,0.456899,45.689944,3.843529,6312.0,8400.0,7378.000000,977.852750,7920.0,7912.0,2688.0,1622.000000,600.0,0.298667,0.180222,0.066667,9000,0.0,9203,0.0,0.0,0.0,0,NaN,1.0,0.693147,1.098612,0.016261,2.125848,1.791759,4.828314,4.394449,5.964350,5.120028,5.406992,0.0,3.344627,1.609438,3.684369,2.386007,4.962285,0.00000,0.000000,5.120028,0.000000,9.105091,NaN,NaN,NaN,9.036106,9.127393,9.036106,8.750366,8.906393,8.977273,8.976262,6.886381,7.896925,6.398595,7.392032,0.261338,0.064539,0.165703,9.036106,8.977273,8.755738,8.750366,NaN,NaN
2,1,3,0.024590,35071,03100200A7,33921,28000,O,37800.0,75.0,NaN,NaN,NaN,C,R,24.58,NArr,BUREDO,C,B,ADF,M0,NaN,DO,335,214,1330.92,615.32,715.60,0.0,265.53,0.0,32.76,0.00,417.31,0.00000,0.00,615.32,0.00,NaN,NaN,NaN,NaN,NaN,NaN,3,122,37800 & 31807 & 18600,75 & 5 & 3,31807.0,18600.0,NaN,NaN,NaN,NaN,0.479857,47.985749,3.891529,18600.0,37800.0,29402.333333,9823.279306,31807.0,18600.0,9400.0,-1402.333333,-9800.0,0.335714,-0.050083,-0.350000,28000,1.0,33921,1.0,1.0,1.0,0,0.0,NaN,0.693147,1.386294,0.024293,3.241811,1.386294,5.817111,5.370638,7.194377,6.423766,6.574518,0.0,5.585487,0.000000,3.519277,0.000000,6.036223,0.00000,0.000000,6.423766,0.000000,10.239996,NaN,NaN,NaN,10.540091,10.431819,10.540091,9.830971,10.288863,10.367473,9.830971,9.192612,9.148571,NaN,NaN,0.289466,-0.430783,-0.051381,10.367473,9.830971,NaN,NaN,NaN,NaN
3,1,4,0.032787,35072,03210323A7,7359,4000,N,NaN,NaN,NaN,NaN

In [308]:
pip install multiprocess

Note: you may need to restart the kernel to use updated packages.


In [309]:
# import libraries

import scipy as sp
import scipy.linalg as spla
import scipy.optimize as spopt
import scipy.stats as spst

import numpy as np
import numpy.linalg as npla


import statsmodels.api as sm


from sklearn.model_selection import KFold

from tqdm import tqdm

from multiprocess import Pool

### Preparation of data for modeling

In [310]:
# choose necessary features
data_model = data_wide[['max_bid', 'res_i', 'estim', 'ln_surf', 'ln_v_total', 'ln_n_trees', 'ln_n_poles', 'stand', 'cut', 'land_area', 'grape', 'quality', 'conditions']]

In [311]:
# drop lines with empty value of 'max_bid'
data_model = data_model.dropna(subset = 'max_bid')

In [312]:
# divide the dataset into factors (X) and target (y)
y = data_model['max_bid']
X_raw = data_model.drop('max_bid', axis = 1)

In [282]:
# add categorical features
X_categ = pd.get_dummies(X_raw[['stand', 'cut', 'land_area', 'grape', 'quality', 'conditions']])
X2 = X_raw.join(X_categ)
X2 = X2.drop(['stand', 'cut', 'land_area', 'grape', 'quality', 'conditions'], axis = 1)

In [283]:
X2

,res_i,estim,ln_surf,ln_v_total,ln_n_trees,ln_n_poles,stand_C,stand_F,stand_T,stand_TSF,cut_A,cut_AUTRE,cut_J,cut_PA,cut_R,land_area_A,land_area_N,land_area_NArr,grape_M0,grape_M1,grape_M2,grape_M3,quality_B,quality_M,quality_MA,quality_MD,quality_TB,conditions_ADD,conditions_ADF,conditions_ADN,conditions_ADTD,conditions_DD
0,24000,25323,2.565718,6.839626,5.849325,5.545177,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
1,9000,9203,2.125848,5.964350,4.828314,4.394449,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0
2,28000,33921,3.241811,7.194377,5.817111,5.370638,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0
5,4500,12710,2.646884,6.615061,5.533389,4.997212,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0
6,24807,24790,3.091497,7.104358,6.516193,4.828314,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,19000,20004,2.342767,6.449554,5.141664,0.693147,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2254,3500,4649,0.693147,5.075861,4.564348,3.044522,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
2255,8855,7653,2.134166,5.591733,4.624973,2.995732,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2256,8695,9536,2.123458,5.812876,4.844187,3.091042,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0


Check, if there are any gaps. 

In [284]:
print(X2.isnull().any().any())

False


As we can see, we don't need filling the gaps. 

In [286]:
# standardize 
X2_norm = (X2 - X2.mean())/X2.std()

### Predictions without regularizations

In [303]:
# function for defining loss without regularization
def loss(y, X, beta):
    err = y - X@beta
    return err@err.T/len(err)

In [294]:
# predict and calculate loss on the sample
total_covs = X2_norm.shape[1]

bounds = [(-100, 100)]*total_covs

print(f'Loss on the sample: {loss(y, X2_norm, beta_opt)}.')

253114859.89224148

In [313]:
# make a model selection
total_covs = X2_norm.shape[1]

bounds = [(-100, 100)]*total_covs

beta_opt = spopt.dual_annealing(lambda b: loss(y, X2_norm, b), bounds).x
print(f'Loss on the sample with Lasso: {loss(y, X2_norm, beta_opt)}.')

Loss on the sample with Lasso: 242601846.1273836.


### Multicollinearity

In [138]:
# plot color scaled correlation matrix for analyzing multicollinearity
corr=data_model.corr()
corr.style.background_gradient(cmap='coolwarm')

,max_bid,res_i,estim,ln_surf,ln_v_total,ln_n_trees,ln_n_poles
max_bid,1.000000,0.936312,0.920087,0.336249,0.673638,0.445205,-0.005308
res_i,0.936312,1.000000,0.964920,0.340504,0.680939,0.433999,-0.027472
estim,0.920087,0.964920,1.000000,0.338397,0.686363,0.428642,-0.060503
ln_surf,0.336249,0.340504,0.338397,1.000000,0.597574,0.626042,0.406662
ln_v_total,0.673638,0.680939,0.686363,0.597574,1.000000,0.824049,0.363302
ln_n_trees,0.445205,0.433999,0.428642,0.626042,0.824049,1.000000,0.584093
ln_n_poles,-0.005308,-0.027472,-0.060503,0.406662,0.363302,0.584093,1.000000


As it can be seen in the correlation matrix, factors are too correlated to each other and this hinders the issuance of a qualitative forecast. 

### Model selection

In [299]:
# define loss with Lasso
def loss2(y, X, beta, mu):
    err = y - X@beta
    return err@err.T/len(err) + mu*np.abs(beta).sum()

In [325]:
# make a model selection
mu = 0.5
total_covs = X2_norm.shape[1]

bounds = [(-100, 100)]*total_covs

beta_opt = spopt.dual_annealing(lambda b: loss2(y, X2_norm, b, mu), bounds).x
print(f'Loss on the sample with Lasso with lambda = 0.5: {loss2(y, X2_norm, beta_opt, mu)}.')

Loss on the sample with Lasso: 242601535.47111908.


In [325]:
# make a model selection
mu = 0.0001
total_covs = X2_norm.shape[1]

bounds = [(-100, 100)]*total_covs

beta_opt = spopt.dual_annealing(lambda b: loss2(y, X2_norm, b, mu), bounds).x
print(f'Loss on the sample with Lasso with lambda = 0.0001: {loss2(y, X2_norm, beta_opt, mu)}.')

Loss on the sample with Lasso: 242601535.47111908.


In this case, Lasso regularization couldn't improve the result: the loss on sample becomes less, when $\mu \rightarrow 0$ (so, the most efficient is $\mu = 0$). 

## Elasticnet

In [164]:
# prepare the split of the sample
folds = 5
kf = KFold(n_splits=folds)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [336]:
# define loss with elasticnet
def loss3(y, X, beta, lamda, mu):
    err = y - X@beta
    return err@err.T/len(err) + lamda*np.square(beta).sum() + mu*np.abs(beta).sum()

In [337]:
# initial params and hyperparams for model
bounds = [(-100, 100)] * X2_norm.shape[1]

lamda = .1
mu = .5

In [338]:
# realization of elasticnet
def worker(param):
    lamda, mu = param
    
    total_out = 0
    total_in = 0
    
    for train_index, test_index in kf.split(X2_norm):
        X_train, X_test = X2_norm[train_index[0] : train_index[-1]], X2_norm[test_index[0] : test_index[-1]]
        y_train, y_test = y[train_index[0] : train_index[-1]], y[test_index[0] : test_index[-1]]
        
        beta = spopt.dual_annealing(lambda b: loss3(y_train, X_train, b, lamda, mu), bounds).x
        reg = lamda*np.square(beta).sum() + mu*np.abs(beta).sum()
        
        err_out = X_test@beta - y_test
        loss_out = err_out@err_out.T/len(err_out)
        total_out += loss_out/folds
        
        err_in = X_train@beta - y_train
        loss_in = err_in@err_in.T/len(err_in) + reg
        total_in += loss_in/folds
        
    print('lamda:{:4}, mu:{:4}, out-of-sample loss:{:7}, in-sample loss:{:7}'.format(lamda, mu, np.round(total_out,4), np.round(total_in,4)))
    

In [339]:
# loss of elasticnet with different hyperparams in-sample and out-of-sample
tasks = []

for lamda in [0, 0.01, 0.02]:
    for mu in [0, 0.01, 0.02]:
        payload = [lamda, mu]
        tasks.append(payload)
        
for t in tqdm(tasks):
    worker(t)

 11%|████▉                                       | 1/9 [01:44<13:58, 104.85s/it]

lamda:   0, mu:   0, out-of-sample loss:243530500.2151, in-sample loss:241865870.8448


 22%|█████████▊                                  | 2/9 [03:26<12:01, 103.05s/it]

lamda:   0, mu:0.01, out-of-sample loss:243530530.1992, in-sample loss:241865902.143


 33%|██████████████▋                             | 3/9 [05:08<10:16, 102.67s/it]

...

lamda:0.02, mu:   0, out-of-sample loss:243538107.2282, in-sample loss:241872058.4573


 89%|███████████████████████████████████████     | 8/9 [14:32<01:52, 112.05s/it]

lamda:0.02, mu:0.01, out-of-sample loss:243538130.4383, in-sample loss:241872089.7266


100%|████████████████████████████████████████████| 9/9 [16:19<00:00, 108.78s/it]

lamda:0.02, mu:0.02, out-of-sample loss:243538152.4082, in-sample loss:241872120.9957


### Conclusion
So, lasso couldn't improve the model and we got the best output with hyperparametr $\mu = 0$; elascticnet regularization gave better result in loss on train sample, but, unfortunately, it didn't help with test sample and the loss remained the same.